<a href="https://colab.research.google.com/github/GianFederico/MD-repo-Natural_Language_Processing/blob/main/NLP_lab5_text_categorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import nltk
nltk.download('names')
from nltk.corpus import names

def gender_features(word):
    return {'last_letter': word[-1]}

def gender_features2(name):
    features = {}
    features["first_letter"] = name[0].lower()
    features["last_letter"] = name[-1].lower()
    for letter in 'abcdefghijklmnopqrstuvwxyz':
        features["count({})".format(letter)] = name.lower().count(letter)
        features["has({})".format(letter)] = (letter in name.lower())
    return features

labeled_names = ([(name, 'male') for name in names.words('male.txt')] +
[(name, 'female') for name in names.words('female.txt')])
import random
random.shuffle(labeled_names)

featuresets = [(gender_features(n), gender) for (n, gender) in labeled_names]
train_set, test_set = featuresets[800:], featuresets[:800] #train/test sets for naive bayes classification
classifier = nltk.NaiveBayesClassifier.train(train_set)

print ("Marco ", classifier.classify(gender_features('Marco')))
print ("Marica ", classifier.classify(gender_features('Marica')))
print ("Andreino ", classifier.classify(gender_features('Andreino')))
print ("Felice ", classifier.classify(gender_features('Felice')))
print ("Felicia ", classifier.classify(gender_features('Felicia')))
print("ACCURACY:", nltk.classify.accuracy(classifier, test_set))

featuresets = [(gender_features2(n), gender) for (n, gender) in labeled_names]
train_set, test_set = featuresets[500:], featuresets[:500]
classifier = nltk.NaiveBayesClassifier.train(train_set)

print("_______________")
print ("Marco ", classifier.classify(gender_features('Marco')))
print ("Marica ", classifier.classify(gender_features('Marica')))
print ("Andreino ", classifier.classify(gender_features('Andreino')))
print ("Felice ", classifier.classify(gender_features('Felice')))
print ("Felicia ", classifier.classify(gender_features('Felicia')))
print("ACCURACY:", nltk.classify.accuracy(classifier, test_set))

Marco  male
Marica  female
Andreino  male
Felice  female
Felicia  female
ACCURACY: 0.7575


[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Package names is already up-to-date!


_______________
Marco  male
Marica  female
Andreino  male
Felice  female
Felicia  female
ACCURACY: 0.772


In [38]:
nltk.download('movie_reviews')
import random

from nltk.corpus import movie_reviews
documents = [(list(movie_reviews.words(fileid)), category)
             for category in movie_reviews.categories()
             for fileid in movie_reviews.fileids(category)]
random.shuffle(documents)
#print (documents[0][1])

all_words = nltk.FreqDist(w.lower() for w in movie_reviews.words())
word_features = list(all_words)[:2000]

def document_features(document):
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains({})'.format(word)] = (word in document_words)
    return features

print(document_features(movie_reviews.words('pos/cv957_8737.txt'))) 

featuresets = [(document_features(d), c) for (d,c) in documents]
train_set, test_set = featuresets[100:], featuresets[:100]
classifier = nltk.NaiveBayesClassifier.train(train_set)

print("ACCURACY:", nltk.classify.accuracy(classifier, test_set))
classifier.show_most_informative_features(8)


[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


{'contains(,)': True, 'contains(the)': True, 'contains(.)': True, 'contains(a)': True, 'contains(and)': True, 'contains(of)': True, 'contains(to)': True, "contains(')": True, 'contains(is)': True, 'contains(in)': True, 'contains(s)': True, 'contains(")': True, 'contains(it)': True, 'contains(that)': True, 'contains(-)': True, 'contains())': True, 'contains(()': True, 'contains(as)': True, 'contains(with)': True, 'contains(for)': True, 'contains(his)': True, 'contains(this)': True, 'contains(film)': False, 'contains(i)': False, 'contains(he)': True, 'contains(but)': True, 'contains(on)': True, 'contains(are)': True, 'contains(t)': False, 'contains(by)': True, 'contains(be)': True, 'contains(one)': True, 'contains(movie)': True, 'contains(an)': True, 'contains(who)': True, 'contains(not)': True, 'contains(you)': True, 'contains(from)': True, 'contains(at)': False, 'contains(was)': False, 'contains(have)': True, 'contains(they)': True, 'contains(has)': True, 'contains(her)': False, 'conta

In [36]:
#regarding the previous cell, use the frequency of a word in a document as a feature instead of simple occurrence

from nltk.corpus import movie_reviews
documents = [(fileid, list(movie_reviews.words(fileid)), category)
             for category in movie_reviews.categories()
             for fileid in movie_reviews.fileids(category)]
random.shuffle(documents)

all_words = nltk.FreqDist(w.lower() for w in movie_reviews.words())
# we took as a vocabulary the 2000 most frequent words in the corpus
word_features = list(all_words)[:2000]

def document_features(document):
    freqDist=nltk.FreqDist(document)
    features = {}
    for word in word_features:
        if word in freqDist:
            features['frequency({})'.format(word)] = freqDist.freq(word)
        else: 
            features['frequency({})'.format(word)] = 0
    return features

istances=[(doc_id,document_features(d),c) for (doc_id, d,c) in documents]

#featuresets = [(document_features(d), c) for (d,c) in documents]
#train_set = featuresets[100:]
#test_set = featuresets[:100]
train_set = [(doc_features,c)
             for (doc_id,doc_features,c) in istances[100:]]
test_set = [(doc_features,c)
             for (doc_id,doc_features,c) in istances[:100]]

classifier = nltk.NaiveBayesClassifier.train(train_set)
print("ACCURACY:", nltk.classify.accuracy(classifier, test_set))
classifier.show_most_informative_features(5)

### looking for errors ###
errors = []
for (doc_id, doc_features, label) in istances[:100]:
    guess = classifier.classify(doc_features)
    if guess != label:
        errors.append( (doc_id, guess, label) )
print ("ERRORS:", len(errors))
print (errors)




#regarding the previous cell, use the number of occurrences of a word in a document as a feature instead of presence/absence of the word
print ("________________________________________________________________________________")
from nltk.corpus import movie_reviews
documents = [(fileid, list(movie_reviews.words(fileid)), category)
             for category in movie_reviews.categories()
             for fileid in movie_reviews.fileids(category)]
random.shuffle(documents)

all_words = nltk.FreqDist(w.lower() for w in movie_reviews.words())
# we took as a vocabulary the 2000 most frequent words in the corpus
word_features = list(all_words)[:2000]

def document_features(document):
    freqDist=nltk.FreqDist(document)
    features = {}
    for word in word_features:
        if word in freqDist:
            features['occurrences({})'.format(word)] = freqDist[word]
        else: 
            features['occurrences({})'.format(word)] = 0
    return features

istances=[(doc_id,document_features(d),c) for (doc_id, d,c) in documents]

#featuresets = [(document_features(d), c) for (d,c) in documents]
#train_set = featuresets[100:]
#test_set = featuresets[:100]
train_set = [(doc_features,c)
             for (doc_id,doc_features,c) in istances[100:]]
test_set = [(doc_features,c)
             for (doc_id,doc_features,c) in istances[:100]]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print("ACCURACY:", nltk.classify.accuracy(classifier, test_set))
classifier.show_most_informative_features(5)

### looking for errors ###
errors = []
for (doc_id, doc_features, label) in istances[:100]:
    guess = classifier.classify(doc_features)
    if guess != label:
        errors.append( (doc_id, guess, label) )
print ("ERRORS:", len(errors))
print (errors)




#regarding the previous cell, use lemmas as features instead of words
print ("________________________________________________________________________________")
import re
from nltk.corpus import stopwords
from nltk.corpus import movie_reviews
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

### apply NLP pipeline to use lemmas as features ###
corpus={}
doc_lemmas={}
all_lemmas=[]
stop_words = stopwords.words('english')

def wordmatch(text):
    cleaned_tokens = re.findall(r"\w+(?:[-']\w+)*|'|[-.(]+|\S\w*", text)
    return cleaned_tokens

def onlypunct(text):
    waste=re.findall(r"[!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~]+", text)
    return waste

for doc_id in movie_reviews.fileids():
        corpus[doc_id]=movie_reviews.raw(doc_id)

for doc_id in corpus.keys():
        tokens=wordmatch(corpus[doc_id])
        waste = onlypunct(corpus[doc_id])
        cleaned_tokens = [t for t in tokens if not t in waste]
        nostop_tokens = [t for t in cleaned_tokens if not t in stop_words]
        norm_tokens = [t.lower() for t in nostop_tokens]
        wnl=nltk.WordNetLemmatizer()
        lemmas = [wnl.lemmatize(t) for t in norm_tokens]
        doc_lemmas[doc_id]=lemmas
        all_lemmas+=lemmas
print ("LEMMAS: ", len(all_lemmas))

####################################################
print ("\npos/cv957_8737.txt LEMMAS:", doc_lemmas['pos/cv957_8737.txt'])

# we took as a vocabulary the 2000 most frequent lemmas in the corpus
lemmasDist = nltk.FreqDist(all_lemmas)
lemma_features = list(lemmasDist)[:2000]

print ("features:", lemma_features)

def document_features(document):
    document_lemmas = set(document)
    features = {}
    for lemma in lemma_features:
        features['contains({})'.format(lemma)] = (lemma in document_lemmas)
    return features
print(document_features(doc_lemmas['pos/cv957_8737.txt']))

documents = [(doc_lemmas[fileid], category)
             for category in movie_reviews.categories()
             for fileid in movie_reviews.fileids(category)]
random.shuffle(documents)

instances = [(document_features(d), c) for (d,c) in documents]
train_set, test_set = instances[100:], instances[:100]

classifier = nltk.NaiveBayesClassifier.train(train_set)

print("ACCURACY:", nltk.classify.accuracy(classifier, test_set))
classifier.show_most_informative_features(5)


ACCURACY: 0.46
Most Informative Features
         frequency(time) = 0.0024630541871921183    pos : neg    =      3.7 : 1.0
           frequency(of) = 0.018867924528301886    pos : neg    =      3.7 : 1.0
          frequency(the) = 0.043478260869565216    pos : neg    =      3.7 : 1.0
            frequency(.) = 0.045454545454545456    neg : pos    =      3.6 : 1.0
            frequency(s) = 0.0055248618784530384    neg : pos    =      3.6 : 1.0
ERRORS: 54
[('neg/cv666_20301.txt', 'pos', 'neg'), ('pos/cv569_26381.txt', 'neg', 'pos'), ('pos/cv048_16828.txt', 'neg', 'pos'), ('pos/cv044_16969.txt', 'neg', 'pos'), ('pos/cv512_15965.txt', 'neg', 'pos'), ('pos/cv768_11751.txt', 'neg', 'pos'), ('pos/cv788_25272.txt', 'neg', 'pos'), ('pos/cv420_28795.txt', 'neg', 'pos'), ('pos/cv650_14340.txt', 'neg', 'pos'), ('pos/cv110_27788.txt', 'neg', 'pos'), ('pos/cv137_15422.txt', 'neg', 'pos'), ('pos/cv508_16006.txt', 'neg', 'pos'), ('neg/cv869_24782.txt', 'pos', 'neg'), ('pos/cv451_10690.txt', 'neg', 'p

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


LEMMAS:  705494

pos/cv957_8737.txt LEMMAS: ['capsule', 'best', 'place', 'start', 'jackie', 'chan', 'newcomer', 'roar', 'along', 'never', 'stop', 'breath', 'frequently', 'hilarious', 'talk', 'jackie', 'chan', 'stuntman', 'miss', 'million', 'thing', 'jackie', 'one', 'accomplished', 'fearsomely', 'unafraid', 'physical', 'adept', 'screen', 'today', 'also', 'funny', 'guy', 'reminds', 'way', 'arnold', 'schwartzenegger', 'used', 'make', 'laugh', "he's", 'funny', 'standing', 'copping', 'look', 'supercop', 'jackie', 'making', 'u', 'laugh', 'scream', 'duck', 'root', 'gasp', 'breath', 'often', 'scene', 'actually', 'third', 'loosely', 'related', 'series', 'movie', 'jackie', 'playing', 'kevin', 'chan', 'hong', 'kong', 'policeman', 'knack', 'getting', 'neck-deep', 'trouble', 'karate-kicking', 'way', 'others', 'also', 'worth', 'seeing', 'little', 'slow', 'start', 'get', 'moving', 'field', 'day', 'movie', 'fairly', 'sophisticated', 'plot', 'movie', 'kind', 'kevin', 'tapped', 'dangerous', 'dea', 'assi

In [39]:
import nltk
import random
nltk.download('brown')

from nltk.corpus import brown
suffix_fdist = nltk.FreqDist()
for word in brown.words():
    word = word.lower()
    suffix_fdist[word[-1:]] += 1  #finding out the most common suffixes
    suffix_fdist[word[-2:]] += 1
    suffix_fdist[word[-3:]] += 1

common_suffixes = [suffix for (suffix, count) in suffix_fdist.most_common(50)]
print("Common suffixes: ",common_suffixes)

def pos_features(word):
    features = {}
    for suffix in common_suffixes:
        features['endswith({})'.format(suffix)] = word.lower().endswith(suffix)
        return features

tagged_words = brown.tagged_words(categories='news')
featuresets = [(pos_features(n), g) for (n,g) in tagged_words]
size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print ("ACCURACY:",nltk.classify.accuracy(classifier, test_set))
print ("apple pos_feature:", classifier.classify(pos_features('apple')))

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


Common suffixes:  ['e', ',', '.', 's', 'd', 't', 'he', 'n', 'a', 'of', 'the', 'y', 'r', 'to', 'in', 'f', 'o', 'ed', 'nd', 'is', 'on', 'l', 'g', 'and', 'ng', 'er', 'as', 'ing', 'h', 'at', 'es', 'or', 're', 'it', '``', 'an', "''", 'm', ';', 'i', 'ly', 'ion', 'en', 'al', '?', 'nt', 'be', 'hat', 'st', 'his']
ACCURACY: 0.17135753356539035
apple pos_feature: AT


In [40]:
import nltk
import random

from nltk.corpus import brown
suffix_fdist = nltk.FreqDist()
for word in brown.words():
    word = word.lower()
    suffix_fdist[word[-1:]] += 1
    suffix_fdist[word[-2:]] += 1
    suffix_fdist[word[-3:]] += 1

common_suffixes = [suffix for (suffix, count) in suffix_fdist.most_common(50)]
print("Common suffixes: ", common_suffixes)

def pos_features(sentence, i):
    features = {"suffix(1)": sentence[i][-1:],
                "suffix(2)": sentence[i][-2:],
                "suffix(3)": sentence[i][-3:]}
    if i == 0:
        features["prev-word"] = "<START>"
    else:
        features["prev-word"] = sentence[i-1]
    return features

# take the first sentence
sent=brown.sents()[0]
print ("An example sentence:", sent)
print ("Features for word: ", sent[8], pos_features(sent, 8))


tagged_sents = brown.tagged_sents(categories='news')
featuresets = []
for tagged_sent in tagged_sents:
    untagged_sent = nltk.tag.untag(tagged_sent)
    for i, (word, tag) in enumerate(tagged_sent):
         featuresets.append( (pos_features(untagged_sent, i), tag) )

size = int(len(featuresets) * 0.1)
train_set, test_set = featuresets[size:], featuresets[:size]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print ("ACCURACY:", nltk.classify.accuracy(classifier, test_set))
sent=['the', 'apple', 'is', 'on', 'the', 'tree']
i=1
print (sent[i], "pos_feature:", classifier.classify(pos_features(sent, i)))


Common suffixes:  ['e', ',', '.', 's', 'd', 't', 'he', 'n', 'a', 'of', 'the', 'y', 'r', 'to', 'in', 'f', 'o', 'ed', 'nd', 'is', 'on', 'l', 'g', 'and', 'ng', 'er', 'as', 'ing', 'h', 'at', 'es', 'or', 're', 'it', '``', 'an', "''", 'm', ';', 'i', 'ly', 'ion', 'en', 'al', '?', 'nt', 'be', 'hat', 'st', 'his']
An example sentence: ['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', "Atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.']
Features for word:  investigation {'suffix(1)': 'n', 'suffix(2)': 'on', 'suffix(3)': 'ion', 'prev-word': 'an'}
ACCURACY: 0.7891596220785678
apple pos_feature: NN
